## 4-3　過去データの結合

In [1]:
import datetime
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta

In [2]:
reserve_tb = pd.read_csv('reserve.csv')
print(reserve_tb.shape)
reserve_tb.head()

(4030, 9)


,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [3]:
customer_tb = pd.read_csv('customer.csv')
print(customer_tb.shape)
customer_tb.head()

(1000, 5)


,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797


In [4]:
month_mst = pd.DataFrame(
    {'year_month': [(datetime.date(2017, 1, 1) + relativedelta(months=x)).strftime('%Y%m') for x in range(0, 3)]}
)
month_mst['join_key'] = 0
month_mst

,year_month,join_key
0,201701,0
1,201702,0
2,201703,0


In [5]:
customer_tb['join_key'] = 0
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,join_key
0,c_1,41,man,35.092193,136.512347,0
1,c_2,38,man,35.325076,139.410551,0
2,c_3,49,woman,35.120543,136.511179,0
3,c_4,43,man,43.034868,141.240314,0
4,c_5,31,man,35.102661,136.523797,0


In [6]:
customer_mst = pd.merge(customer_tb[['customer_id', 'join_key']], month_mst, on='join_key')
customer_mst.head()

,customer_id,join_key,year_month
0,c_1,0,201701
1,c_1,0,201702
2,c_1,0,201703
3,c_2,0,201701
4,c_2,0,201702


In [7]:
reserve_tb['year_month'] = reserve_tb['checkin_date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d').strftime('%Y%m'))
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,year_month
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,201603
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,201607
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,201610
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,201703
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,201709


In [8]:
summary_result = pd.merge(
    customer_mst,
    reserve_tb[['customer_id', 'year_month', 'total_price']],
    on=['customer_id', 'year_month'],
    how='left'
)

print(summary_result.shape)
summary_result.head()

(3012, 4)


,customer_id,join_key,year_month,total_price
0,c_1,0,201701,NaN
1,c_1,0,201702,NaN
2,c_1,0,201703,194400.0
3,c_2,0,201701,NaN
4,c_2,0,201702,NaN


In [9]:
summary_result = summary_result.groupby(['customer_id', 'year_month'])['total_price'].sum().reset_index()
summary_result.fillna(0, inplace=True)

print(summary_result.shape)
summary_result.head()

(3000, 3)


,customer_id,year_month,total_price
0,c_1,201701,0.0
1,c_1,201702,0.0
2,c_1,201703,194400.0
3,c_10,201701,0.0
4,c_10,201702,0.0
